In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [4]:
df = pd.read_csv("./data/datos_nasdaq.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Fecha     504 non-null    object
 1   Último    504 non-null    object
 2   Apertura  504 non-null    object
 3   Máximo    504 non-null    object
 4   Mínimo    504 non-null    object
 5   Vol.      503 non-null    object
 6   % var.    504 non-null    object
dtypes: object(7)
memory usage: 27.7+ KB


In [6]:
df.describe()

,Fecha,Último,Apertura,Máximo,Mínimo,Vol.,% var.
count,504,504,504,504,504,503,504
unique,504,504,504,504,503,492,348
top,30.11.2023,"15.947,87","15.986,40","16.011,98","11.235,74","249,11M","-1,10%"
freq,1,1,1,1,2,2,5


In [8]:
df.head()

,Fecha,Último,Apertura,Máximo,Mínimo,Vol.,% var.
0,30.11.2023,"15.947,87","15.986,40","16.011,98","15.825,83","444,54M","-0,25%"
1,29.11.2023,"15.987,60","16.111,59","16.166,18","15.976,69","230,98M","-0,14%"
2,28.11.2023,"16.010,43","15.939,12","16.031,38","15.921,45","238,59M","0,30%"
3,27.11.2023,"15.961,98","15.971,05","16.045,30","15.933,04","215,45M","-0,11%"
4,24.11.2023,"15.980,11","15.972,89","16.007,42","15.940,62","68,57M","-0,13%"


In [9]:
col = ['Último', 'Apertura', 'Máximo', 'Mínimo']

for columna in col:
    df[columna] = df[columna].str.replace('.', '').str.replace(',', '.').astype(float)


In [10]:
df.dropna(inplace=True)

In [11]:
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d.%m.%Y')

In [12]:
df['Vol.'] = df['Vol.'].str.replace(',', '')

# Identificar las filas que contienen 'M'
m = df['Vol.'].str.endswith('M') & ~df['Vol.'].isna()

# Reemplazar 'M' por '', convertir a float y multiplicar por 1 millón
df.loc[m, 'Vol.'] = df.loc[m, 'Vol.'].str.replace('M', '').astype(float) * 1_000_000

# Convertir a float
df['Vol.'] = df['Vol.'].astype(float)



In [13]:
df['% var.'] = df['% var.'].str.rstrip('%').str.replace(',', '.').astype(float)

In [98]:
df

,Fecha,Último,Apertura,Máximo,Mínimo,Vol.,% var.
0,2023-11-30,15947.87,15986.40,16011.98,15825.83,4.445400e+10,-0.25
1,2023-11-29,15987.60,16111.59,16166.18,15976.69,2.309800e+10,-0.14
2,2023-11-28,16010.43,15939.12,16031.38,15921.45,2.385900e+10,0.30
3,2023-11-27,15961.98,15971.05,16045.30,15933.04,2.154500e+10,-0.11
4,2023-11-24,15980.11,15972.89,16007.42,15940.62,6.857000e+09,-0.13
...,...,...,...,...,...,...,...
499,2021-12-06,15846.16,15744.91,15898.59,15557.78,2.445300e+10,0.85
500,2021-12-03,15712.04,16039.41,16089.61,15543.31,2.876900e+10,-1.74
501,2021-12-02,15990.76,15791.30,16066.32,15770.92,2.521000e+10,0.71
502,2021-12-01,15877.72,16347.96,16427.19,15864.00,2.535500e+10,-1.60


In [14]:
columnas = ['Último', 'Apertura', 'Máximo', 'Mínimo', 'Vol.', '% var.']

In [15]:
print(df.dtypes)

Fecha       datetime64[ns]
Último             float64
Apertura           float64
Máximo             float64
Mínimo             float64
Vol.               float64
% var.             float64
dtype: object


In [16]:
def describe_df(df):
    tipo_de_dato = df.dtypes
    valores_nulos = (df.isna().mean() * 100).round(2)
    valores_unicos = df.nunique()
    cardinalidad = ((df.nunique()/df.shape[0])*100).round(2)

    descripcion = pd.DataFrame({
        'Tipo de dato': tipo_de_dato,
        'Valores nulos (%)': valores_nulos,
        'Valores unicos': valores_unicos,
        'Cardinalidad (%)': cardinalidad
    })

    return descripcion
    
    

In [17]:
describe_df(df).T

,Fecha,Último,Apertura,Máximo,Mínimo,Vol.,% var.
Tipo de dato,datetime64[ns],float64,float64,float64,float64,float64,float64
Valores nulos (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0
Valores unicos,503,503,503,503,502,492,348
Cardinalidad (%),100.0,100.0,100.0,100.0,99.8,97.81,69.18


In [43]:
def tipifica_variables(df, umbral_categoria = 5, umbral_continua = 10.5):
    df_final = pd.DataFrame(columns=['nombre_variable', 'tipo_sugerido'])

    for col in df.columns:
        columna = df[col]

        cardinalidad = columna.nunique()
        cardi_porcen = cardinalidad/len(columna)

        if cardinalidad == 2:
            tipo = 'Binaria'
        elif cardinalidad < umbral_categoria:
            tipo = 'Categorica'
        elif cardi_porcen >=  umbral_continua:
            tipo = 'Numerica continua'
        else:
            tipo = 'Numerica discreta'

        df_final = pd.concat([df_final, pd.DataFrame({'nombre_variable': [col], 'tipo_sugerido': [tipo]})], ignore_index=True)

    return df_final

In [44]:
tipifica_variables(df, 'Apertura', 'Último')

TypeError: '<' not supported between instances of 'int' and 'str'

### Funcion: get_features_num_regression

Esta función recibe como argumentos un dataframe, el nombre de una de las columnas del mismo (argumento 'target_col'), que debería ser el target de un hipotético modelo de regresión, es decir debe ser una variable numérica continua o discreta pero con alta cardinalidad, además de un argumento 'umbral_corr', de tipo float que debe estar entre 0 y 1 y una variable float "pvalue" cuyo valor debe ser por defecto "None".

La función debe devolver una lista con las columnas numéricas del dataframe cuya correlación con la columna designada por "target_col" sea superior en valor absoluto al valor dado por "umbral_corr". Además si la variable "pvalue" es distinta de None, sólo devolvera las columnas numéricas cuya correlación supere el valor indicado y además supere el test de hipótesis con significación mayor o igual a 1-pvalue.

La función debe hacer todas las comprobaciones necesarias para no dar error como consecuecia de los valores de entrada. Es decir hará un check de los valores asignados a los argumentos de entrada y si estos no son adecuados debe retornar None y printar por pantalla la razón de este comportamiento. Ojo entre las comprobaciones debe estar que "target_col" hace referencia a una variable numérica continua del dataframe.

In [114]:
from scipy.stats import pearsonr

def get_features_num_regression(df, target_col, umbral_corr, pvalue=None):
    # Verificamos si existe algún error al llamar a la columna 'target_col'
    if target_col not in df.columns:
        print(f"Error: La columna '{target_col}' no está bien indicada, no se puede asignar como 'target_col'.")
        return None

    # Verificamos si 'target_col' es una variable numérica continua
    if not np.issubdtype(df[target_col].dtype, np.number):
        print(f"Error: La columna '{target_col}' no es una variable numérica continua.")
        return None

    # Verificamos si 'umbral_corr' está en el rango [0, 1]
    if not (0 <= umbral_corr <= 1):
        print("Error: Se ha indicado un 'umbral_corr' incorrecto, debe estar entre el rango [0, 1].")
        return None
    
    # Verificamos si 'pvalue' es un valor válido
    if pvalue is not None and (not isinstance(pvalue, (float, int)) or pvalue <= 0 or pvalue >= 1):
        print("Error: Si 'pvalue' no es 'None', debe tener un valor entre (0, 1).")
        return None

    # Obtenemos las columnas numéricas del dataframe
    cols_numericas = df.select_dtypes(include=[np.number]).columns

    # Calculamos la correlación y p-value para cada columna numérica con 'target_col'
    correlaciones = []
    for col in cols_numericas:
        corr, p_value = pearsonr(df[col], df[target_col])
        correlaciones.append((col, corr, p_value))

    # Filtramos las columnas basadas en 'umbral_corr' y 'pvalue'
    features_seleccionadas = []
    for col, corr, p_value in correlaciones:
        if abs(corr) > umbral_corr and (pvalue is None or p_value < 1 - pvalue):
            features_seleccionadas.append((col, corr, p_value))

    # Devolvemos la lista de características seleccionadas junto con sus correlaciones y valores p
    return features_seleccionadas


In [118]:
get_features_num_regression(df, 'Mínimo', 0.3, pvalue=0.1)


[('Último', 0.9972805808746535, 0.0),
 ('Apertura', 0.9973535582132576, 0.0),
 ('Máximo', 0.9972570817269241, 0.0),
 ('Mínimo', 0.9999999999999999, 0.0)]

In [ ]:
def tipifica_variables(df, umbral_categoria, umbral_continua):
    df_final = pd.DataFrame(columns=['nombre_variable', 'tipo_sugerido'])

    for col in df.columns:
        columna = df[col]

        cardinalidad = columna.nunique()
        cardi_porcen = cardinalidad/len(columna)

        if cardinalidad == 2:
            tipo = 'Binaria'
        elif cardinalidad < umbral_categoria:
            tipo = 'Categorica'
        elif cardi_porcen >=  umbral_continua:
            tipo = 'Numerica continua'
        else:
            tipo = 'Numerica discreta'

        df_final = df_final.append({'nombre_variable': col, 'tipo': tipo_sugerido}, ignore_index=True)

    return df_final

In [ ]:
def plot_features_cat_regression(df, target_col="", columns[], pvalue=0.05, with_individual_plot=False):

    if not isinstance(df, pd.DataFrame) or df.empty:
        return ValueError("El dataframe está vacío")
    if target_col not in df.columns:
        return ValueError("El target no está en el dataframe")
    if not columns:
        columns = df.select_dtypes(inlude=['object'].columns)
    else:
        for col in columns:
            if col not in df.columns:
                return ValueError("Esta columna no está en el dataframe")    
    if not 0 < pvalue < 1:
        return ValueError("El valor de pvalue no es correcto")
    
    
    if with_individual_plot:
        for col in x:    
        plt.figure(figsize=(10, 8))
        sns.barplot(x='target_col', y='col', data=df)
        plt.xlabel('Target')
        plt.ylabel("Col")
        plt.title("Relación entre target y 'col'")

        plt.tight_layout()
        plt.show()